# Künstliche Neuronale Netze
In diesem Notebook wird ein einfaches künstliches Neuronales Netz zur Klassifizierung von handgeschriebenen Zahlen implementiert und demonstriert. Die Deep-Learning Bibliothek [Keras](https://keras.io/) wird genutzt, um für Einstieg in die Programmierung auf einer hohen Abstraktionsebene zu ermöglichen. Keras setzt auf dem bekannten Deep Learning Framework [TensorFlow](https://tensorflow.org) auf.

![MNIST dataset of handwritten digits](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

Der [MNIST-Datensatz](http://yann.lecun.com/exdb/mnist/) ist ein sehr beliebtes Beispiel, um anhand eines einfachen Klassifizierungsproblems Machine Learning Techniken zu demonstrieren. Die 60000 Beispiele dieses Datensatzes sind Bilder von handgeschriebenen Zahlen in der Auflösung 28x28-Pixel und den dazugehörigen beschreibenden Labels (z.B. "2", "9", ...). Aufgrund der begrenzten Rechenkapazität wird werden die Modelle hier mit einer Teilmenge von 2000 Bildern trainiert und mit 1000 Bildern getestet.

## Programmbiblioteken importieren
Neben unterschiedlichen Modulen der Keras-Bibliothek wird `matplotlib` für die Visualisierungen importiert.

In [ ]:
import keras
import helper
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

## Datensatz laden
Keras bringt eine [Reihe von Datensätzen](https://keras.io/datasets/) zum Ausprobieren der Bibliothek mit. Mit einem einzigen Funktionsaufruf kann wie folgend der Test- und Trainingsdatensatz in entsprechende Varablen geladen werden.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Daten überprüfen
Mit Ausführen der nächsten Codezelle werden probeweise neun handgeschriebene Zahlen aus dem Datensatz mitsamt der dazugehörigen Labels ausgegeben.

In [ ]:
helper.plot_images(x_train[0:9], y_train[0:9])

## Daten vorverarbeiten
Um die zweidimensionalen Bilder in ein einfaches neuronales Netz geben zu können, müssen diese in Vektoren umgeformt werden. Aus 28x28 Pixel Bildern werden 784-dimensionale Vektoren.

In [ ]:
# Anzahl der Klassen festlegen
num_classes = 10
num_train_images = 2000

x_train = x_train[0:num_train_images].reshape(-1, 784).astype('float32')
x_test = x_test.reshape(-1, 784).astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Kodieren der Klassenlabels nach dem One-Hot Prinzip
y_train = keras.utils.to_categorical(y_train, num_classes)[0:num_train_images]
y_test = keras.utils.to_categorical(y_test, num_classes)

## Modellarchitektur

![Netzarchtektur für MNIST-Klassifizierung](https://3.bp.blogspot.com/-mDyzBzA4btg/V4_Z0f2mc7I/AAAAAAAAE3M/dtU8hT661fQWtnRC_JvIH_4qifQomZ4PACLcB/s1600/MNIST_neuralnet_image.png)

In der folgenden Codezelle die Architektur des neuronalen Netzes definiert. In dem `Sequential(...)`-Funktionsaufruf befindet sich eine Liste von Anweisungen, die von Input bis Output das Netz konstruieren. `Dense(100)` bedeutet beispielsweise, dass eine Schicht mit 100 Neuronen in das Modell eingefügt wird. Der darauffolgende Aufruf `Activation('sigmoid')` bedeutet, dass die Sigmoid-[Aktivierungsfunktion](https://keras.io/activations/) genutzt wird.  

Die `softmax`-Aktivierungsfunktion am Ende des Modells normiert die Ausgabewerte der letzten Neuronenschicht, sodass diese in der Summe 1 ergeben - so kann das Ergebnis als Wahrscheinlichkeit interpretiert werden. Jedes Output-Neuron steht für eine Klasse der handgeschriebenen Zeichen.

Das Modell kann beliebig erweitert und verändert werden.

In [ ]:
model = Sequential([
    Dense(100, input_shape=(784,)),
    Activation('sigmoid'),
    #Dense(100),
    #Activation('sigmoid'),
    Dense(10),
    Activation('softmax'),
])

Der folgenden Aufruf `model.compile(...)` weist dem Modell eine Fehlerfunktion (`loss`) und einen Optimizer (vgl. Gradient Descent) zu. Nach Ausführen der folgenden Codezeile kann das Modell trainiert werden.

In [ ]:
# Gegebenenfalls bereits vorhandene Gewichte löschen.
model.reset_states()

# Optimizer und Loss festlegen und das Modell kompilieren.
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Modell trainieren
Nun wird die Anzahl der Durchläufe durch den Datensatz zum Trainieren festgelegt. Das Ausführen der nächsten Zelle startet den Trainingsvorgang.

In [ ]:
# Anzahl der Iterationen durch den kompletten Datensatz (Epochs)
num_epochs = 10

# Trainingsvorgang des Modells mit einer batch-size von 32 starten
training_history = model.fit(x_train, y_train, epochs=num_epochs, batch_size=32, validation_data=(x_test[0:1000], y_test[0:1000]))

## Modell evaluieren
Es kann hilfreich sein, die Entwicklung der Fehlermetriken über den Trainingsverlauf zu beobachten. Während die Fehlermetrik `loss` den Durchschnittlichen Fehler während des Trainings auf dem Trainingsdatensatz beschreibt, wird der sogenannte _Validation-Loss_ nach jedem Interation durch den Datensatz (_Epoch_) berechnet: `val_loss` ist der Fehler auf einer Teilmenge des Test-Datensatzes. Konvergiert die Auswertung der Trainings-Lossfunktion gegen 0, ist die gesuchte Funktion für die gewählte Netzarchitektur lernbar. Auch die Entwicklung des Validation-Loss ist wichtig.

In [ ]:
plt.plot(training_history.history['loss'])
plt.plot(training_history.history['val_loss'])
plt.title('Loss-Entwicklung über den Trainingsverlauf')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(x_test, y_test)

### Experimente
* Was könnte passieren, wenn der Parameter `batch_size` zu klein bzw. zu groß gewählt ist?
* Welches Problem könnte vorliegen, wenn in obiger Abbildung die blaue Kurve des Trainings Loss-Verlaufs gegen 0 konvergiert, die orange Validation-Loss Kurve aber nach vielen viele Datensatz-Iterationen (`epochs`) nach oben ausschlägt?
* Die während der Vorverarbeitung definierte Variable `num_train_images` gibt die Anzahl der für das Training zu verwendenden Beispiele an. Wie verhält sich die Genauigkeit der Klassifizierung, wenn die Anzahl verringert wird?
* In der Modellarchitektur in der Funktion `model.compile` wird `rmsprop` als Optimizer gesetzt. Die Keras-Dokumentation führt eine [Liste von Optimizern](https://keras.io/optimizers/). Welche Auswirkungen hat die Wahl des Optimizers auf den Trainingsverlauf bzw. die Ergebnisse?
* Im ersten Codeblock der Modellarchitektur befinden sich zwei auskommentierte Zeilen, die mit einer Raute beginnen. Werden die Rauten dieser beiden Zeilen entfernt, vergrößert sich das Modell um eine weitere Schicht. Nun kann die Zelle und alle darauffolgenden Zellen erneut ausgeführt werden. Verändert sich die Genauigkeit des Modells auf dem Testdatensatz? Auch die Anzahl der Neuronen (`100`) der Ebenen und die Aktivierungsfunktion der Neuronen kann an dieser Stelle geändert werden.

## Weiterführende Links

[TensorFlow Playground](https://playground.tensorflow.org/) ist eine Browserbasierte Simulation eines künstlichen neuronalen Netzes. Es können verschiedene Datensätze ausgewählt, die Modellarchitektur angepasst und der Trainingsprozess überwacht werden.